# Generación de datos sintéticos con *Generative Adversarial Network* (GAN)

En este cuaderno se muestra un ejemplo de creación de datos sintéticos utilizando  *Generative Adversarial Network* (GAN) en el lenguaje Python. 
El método aquí presentado está basado en el documento [Modeling Tabular Data using Conditional GAN](https://arxiv.org/pdf/1907.00503)

A un nivel muy general, las GAN funcionan creando dos modelos, 

- Un modelo que transforma datos aleatorios en vectores semejantes a un registro. Al que se le llamara generador
- Un modelo que al recibir un vector semejante a un registro, decide si este registro es real o sintético, llamado discriminador

Estos dos modelos son entrenados iterativamente con distintos propósitos, el discriminador es entrenado para poder diferenciar entre los registros reales y los creados por el generador. 
Por otra parte el generador es entrenado para crear registros tan auténticos que logren engañar al discriminador.


## Instalación e importación de bibliotecas 

Para este ejercicio se hace uso de la biblioteca [SDV](https://sdv.dev/SDV/index.html) la cual permite generar datos sintéticos.

También se hace uso de la biblioteca pandas para el manejo de los datos en el lago de datos.


Este cuaderno está basado en la imagen de contenedor [jupyter/tensorflow-notebook:python-3.8](https://hub.docker.com/layers/jupyter/tensorflow-notebook/python-3.8/images/sha256-c077d121c38b1435ed7a9652180eae21e6d92ad842cc2a53e7162d3719604cfb?context=explore)

In [ ]:
pip install s3fs==2022.11.0 sdv==0.17.1 kaleido

In [2]:
from getpass import getpass
import pandas as pd 

from sdv.tabular import CTGAN
from sdv.evaluation import evaluate

from sdmetrics.reports.single_table import QualityReport

from sdmetrics.reports import utils

## Acceso a los datos del lago

Para este ejercicio se obtuvieron datos de la Encuesta de Ocupación y Empleo. Estos difieren de los datos disponibles públicamente ya que contienen información real de localidad y manzana que son omitidas por cuestiones de privacidad. 

### Variables de acceso al lago 

En la siguiente celda se establecen variables para acceder al lago de datos. Es necesario tener accesos adecuados para obtener los datos. 

In [ ]:
usuario_lago = input("Usuario: ")
contraseña_lago =  getpass(prompt = 'Contraseña: ')

In [ ]:
opciones_almacenamiento={
        "key": usuario_lago,
        "secret": contraseña_lago,
        "client_kwargs": {"endpoint_url": "http://host:port"}
    }

In [ ]:
ruta_archivo = "s3://carpeta/archivo.ext"

tabla_sdem = pd.read_csv(
    ruta_archivo, 
    storage_options = opciones_almacenamiento,
    encoding = "latin-1",
    low_memory=False,
    dtype= {
        'LOC':"str", 'MUN':"str", 'T_LOC':"str", 'MAN':"str", 'CD_A':"str", 'ENT':"str", 'AGEB':"str", 
        'SEX':"str", 'EDA':'float32', 'NAC_DIA':"str", 'NAC_MES':"str", 'NAC_ANIO':'float32', 'CS_P13_1':"str", 'POS_OCU':"str", 'INGOCUP':'float32',
       'FAC':'float32'
    },
    na_values = [" ",""]
)

## Variables de interés 

Para este ejercicio no requerimos hacer uso de FAC el cual indica el factor de expansión. Por lo que las variables de interés serán:

- **LOC:** Localidad
- **MUN:** Municipio
- **T_LOC:** Tamaño de localidad 
- **MAN:** Manzana
- **CD_A:** Ciudad auto representada
- **ENT:** Entidad
- **SEX:** Sexo
- **EDA:** Edad 
- **NAC_DIA:** Dia de nacimiento
- **NAC_MES:** Mes de nacimiento
- **NAC_ANIO:** Año de nacimiento 
- **CS_P13_1:** Nivel escolar
- **POS_OCU:** Posición en la ocupación
- **INGOCUP:** Ingreso del personal ocupado 

Como se puede observar, las variables son en general variables de identificación o variables sensibles. Por lo que la aplicación de datos sintéticos resulta ideal.

In [6]:
variables_interes = ['LOC','MUN','T_LOC','MAN','CD_A','ENT','SEX','EDA','NAC_DIA','NAC_MES','NAC_ANIO','CS_P13_1','POS_OCU','INGOCUP']

tabla_sdem_interes = tabla_sdem[variables_interes]

In [7]:
metadatos = {
    "fields":{
        'LOC':{"type": "categorical"},
        'MUN':{"type": "categorical"},
        'T_LOC':{"type": "categorical"},
        'MAN':{"type": "categorical"},
        'CD_A':{"type": "categorical"},
        'ENT':{"type": "categorical"},
        'SEX':{"type": "categorical"},
        'EDA':{"type": "numerical", 'subtype': 'integer'},
        'NAC_DIA':{"type": "categorical"},
        'NAC_MES':{"type": "categorical"},
        'NAC_ANIO':{"type": "numerical", 'subtype': 'integer'},
        'CS_P13_1':{"type": "categorical"},
        'POS_OCU':{"type": "categorical"},
        'INGOCUP':{"type": "numerical", 'subtype': 'integer'},
    },
    'constraints': [],
    'model_kwargs': {},
    'name': None,
    'primary_key': None,
    'sequence_index': None,
    'entity_columns': [],
    'context_columns': []
}


## Creación del modelo 

Creamos el modelo, el cual fue adaptado para reducir el tiempo de entrenamiento.
Adicionalmente medimos el tiempo que tomó hacer el entrenamiento. 

Aun con las medidas tomadas los tiempos de entrenamiento se pueden extender a horas por lo que debe usarse con precaución.

In [ ]:
model = CTGAN(
        epochs=10,
    batch_size=100,
    verbose=True
)

In [ ]:
%%time
model.fit(
    tabla_sdem_interes

)

In [ ]:
model.save('GAN_ENOE.pkl')

### Creación de datos sintéticos 

Ya que esta el modelo creado, se puede utilizar para crear registros sintéticos.

In [8]:
model = CTGAN.load('GAN_ENOE.pkl')

In [ ]:
%%time
tabla_sdem_interes_syn = model.sample(len(tabla_sdem_interes))
tabla_sdem_interes_syn

## Revisión de resultados

La biblioteca SDG también proporciona métodos para evaluar los datos generados.

Podemos obtener un dato general el cual es simplemente el agregado de las métricas calculadas, las cuales también se pueden obtener como se aprecia debajo.


In [ ]:
evaluate(tabla_sdem_interes_syn, tabla_sdem_interes)

In [ ]:
evaluate(tabla_sdem_interes_syn, tabla_sdem_interes,aggregate =False)

Es posible obtener un reporte más detallado con información por columna.

Para columnas sencillas se evalúan las distribuciones marginales. Dependiendo de la naturaleza de la columna se evalúa con 
- el estadístico Kolmogorov-Smirnov complemento para variables continuas
- la distancia de variación total (TVD por sus singlas en inglés) para variables categóricas

También se puede evaluar pares de columnas calculando la similitud en la correlación o la similitud en las tablas de contingencia.

Para ello es necesario generar un reporte de calidad como se muestra a continuación


In [ ]:
reporte = QualityReport()

reporte.generate(tabla_sdem_interes, tabla_sdem_interes_syn, metadatos)

In [ ]:
reporte.get_details(property_name='Column Shapes')

In [ ]:
reporte.get_details(property_name='Column Pair Trends')

Finalmente, podemos evaluar de manera visual la distribución de ambas tablas para detectar anomalías

In [ ]:
for columna in tabla_sdem_interes.columns:
    utils.get_column_plot(
        tabla_sdem_interes,
        tabla_sdem_interes_syn,
        column_name=columna,
        metadata=metadatos
    ).show(renderer="png")